In [547]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by D&D Optimised

## Calculate average damage

### Source: D&D Optimised

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

In [548]:
Critit=1/20

In [549]:
Critit

0.05

### Critical hit with advantage

In [550]:
CrititA=1-(Critit)**2

In [551]:
CrititA

0.9975

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

In [552]:
1- (1-1/20)**3

0.1426250000000001

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [553]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [554]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight.

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [555]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [556]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [557]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [558]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [559]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [560]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [561]:
Level=8;

### Calculate the Proficiency Bonus

In [562]:
ProficiencyBonus=1+np.ceil(Level/4)

In [563]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [564]:
OtherBonus=1; #Rod of pact keeper

In [565]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [566]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [567]:
WeaponBonus=1; #Magical +1 weapon

In [568]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [569]:
MeleeHit

8

In [570]:
### Calculate Hit for Weapon Attacks with Dexterity

In [571]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [572]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [573]:
OAC=23

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [574]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [575]:
PrHit

0.30000000000000004

#### Probability to miss the Opponent (for 1 dice roll)

In [576]:
PrMiss=(OAC- SpellHit -1)/20 

In [577]:
PrMiss

0.7

#### Probability to hit the opponent with advantage (Ad=2)

In [578]:
Ad=2

In [579]:
PrHitA=1-(PrMiss)**(Ad)

In [580]:
PrHitA

0.51

#### Number of Elrdich Blast Rays

In [581]:
Rays=min(math.floor(Level/5)+1,4)

In [582]:
Rays

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [583]:
AverageDamagePerRay=5+Modifiers[Abilities.index('CHA')]

In [584]:
AverageDamagePerRay

9

Additional modifier for Hex spell

In [585]:
HexAverageDamage=3

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [586]:
2 * PrHit * PrMiss * AverageDamagePerRay+ 2* (Critit) *(1-Critit) * AverageDamagePerRay+ PrHit**2 * 2* AverageDamagePerRay + 2* (Critit) *(Critit) * AverageDamagePerRay

6.300000000000001

In order to easily generalise to higher number of rays we use the binomial distribution. we check for 2 Rays to see the same result as above.

Without taking into account Crit Hit

In [587]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerRay 

In [588]:
ADNCR

5.400000000000002

Calculating Crit Hit

In [589]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerRay 

In [590]:
ACHD

0.9000000000000002

Adding the two together

In [591]:
TAD=ADNCR+ACHD

In [592]:
TAD

6.3000000000000025

#### Calculating for arbitrary number of Rays

In [593]:
TotalDamage=0;

In [594]:
indexRay = 1
while indexRay < (Rays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,Rays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,Rays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [595]:
TotalDamage

6.3000000000000025

### Average damage with  Advantage

In [596]:
n=1

In [597]:
TotalRays=n*Rays

In [598]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [599]:
TotalDamage

33.435

### Average damage with Advantage and Hex

In [600]:
n=1

In [601]:
TotalRays=n*Rays

In [602]:
TotalDamage=0

In [603]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [604]:
TotalDamage

36.18

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [605]:
n=2;

In [606]:
TotalRays=n*Rays

In [607]:
TotalDamage=0

In [608]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [609]:
TotalDamage

12.600000000000009

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [610]:
n=2;

In [611]:
TotalRays=n*Rays

In [612]:
TotalDamage=0

In [613]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [614]:
  TotalDamage

54.27

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [615]:
n=2;

In [616]:
TotalRays=n*Rays

In [617]:
TotalDamage=0

In [618]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [619]:
TotalDamage

72.36000000000001

## Calculate the damage of Melee + Hex + Smite with Advantage

In [620]:
WeaponDamage=9 #Longsword +1

In [621]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [622]:
PrHit

0.30000000000000004

In [623]:
PrMiss=1-PrHit

In [624]:
Ad=2

In [625]:
PrHitAd=1-(PrMiss)**(Ad)

In [626]:
PrHitAd

0.51

In [627]:
Smite=20

In [628]:
BonusSmite=6

In [629]:
TotalDamage= (WeaponDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [630]:
TotalDamage

57.285